In [1]:
%run constants.ipynb
%run gen_h.ipynb

In [2]:
import gurobipy as gp
from gurobipy import GRB
from itertools import repeat
import re

In [3]:
def var_name(*ineqs, prefix=''):
    return prefix + f'{str(ineqs)}'

In [4]:
def extract_ineq(var):
    match = re.findall(r'\(([^()]+)\)', var.varName)
    return sympify(match[0]) if match else None

In [5]:
def extract_ineqs(variables):
    ineqs = []
    for v in variables:
        e = extract_ineq(v)
        if v.x == 1 and e is not None:
            ineqs.append(e)
        elif v.x == 0:
            # indicator is false
            ineqs.append(Not(And(e)))
    return ineqs

In [6]:
def to_gp_linexpr(sympy_expr, var_map):
    gurobi_expr = gp.LinExpr()

    if isinstance(sympy_expr, Add):
        for term in sympy_expr.args:
            gurobi_expr.add(to_gp_linexpr(term, var_map))
    elif sympy_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sympy_expr.args:
            if factor.is_number:
                coeff *= float(factor)
            elif factor in var_map:
                var = var_map[factor]
        if var is not None:
            gurobi_expr.addTerms(coeff, var)
        else:
            gurobi_expr.addConstant(coeff)
    elif sympy_expr in var_map:
        gurobi_expr.addTerms(1.0, var_map[sympy_expr])
    elif sympy_expr.is_number:
        gurobi_expr.addConstant(float(sympy_expr))

    return gurobi_expr

In [7]:
EPS = 1
def unify_ineq(ineq):
    '''
    Converts the given inequality to the form expr >= 0.
    '''
    lhs, rhs, op = ineq.lhs, ineq.rhs, ineq.rel_op
    # Check the inequality type and unify the format
    if op == '>=':
        # lhs >= rhs -> lhs - rhs >= 0
        return Rel(lhs - rhs, 0, '>=')
    elif op == '>':
        # lhs > rhs -> lhs - rhs > 0 => lhs - rhs >= EPS
        return Rel(lhs - rhs - EPS, 0, '>=')
    elif op == '<=':
        # lhs <= rhs -> lhs - rhs <= 0 -> rhs - lhs >= 0
        return Rel(rhs - lhs, 0, '>=')
    elif op == '<':
        # lhs < rhs -> rhs > lhs -> rhs - lhs > 0 -> rhs - lhs >= EPS
        return Rel(rhs - lhs - EPS, 0, '>=')
    else:
        raise ValueError(f"Unhandled inequality type: {op}")

In [17]:
def to_gp_ineq(ineq, var_map):
    return (to_gp_linexpr(unify_ineq(ineq).lhs, var_map) >= 0)

In [18]:
def add_ind_constr(ineq, model=None, name='ind', var_map=None):
    ''' Add indicator constraint for a single inequality'''
    r = unify_ineq(ineq)  # to the form >= 0
    lhs = to_gp_linexpr(r.lhs, var_map)
    v = model.addVar(vtype=GRB.BINARY, name=name)
    model.addConstr((v == 1) >> (lhs >= 0))
    model.addConstr((v == 0) >> (lhs <= -EPS))  # lhs < 0
    return v

In [19]:
# for each compound indicator group, restrict their sum to be [0,1]
def add_ind_constrs(*ineqs, model=None, var_map=None):
    ''' Adds variables and constraints for a compound indicator. '''
    if len(ineqs) == 1:
        return (add_ind_constr(ineqs[0], 
                               name=var_name(ineqs[0], prefix='single_ind'), 
                               var_map=var_map,
                               model=model),)
    
    I = model.addVar(vtype=GRB.BINARY, 
                     name=var_name(*ineqs, prefix='main_ind'))
    component_vars = [add_ind_constr(i, 
                                     name=var_name(i, prefix='component_ind'), 
                                     var_map=var_map,
                                     model=model) for i in ineqs]
    model.addConstr(I == gp.and_(component_vars), 
                    name=var_name(*ineqs, prefix='andconstr'))
    return I, component_vars

In [20]:
def analyse_result(model):
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found:")
        for v in model.getVars():
            print(f"{v.varName}: {v.x}, {not (not v.x)}")
        print(f"Objective value: {model.objVal}")
    elif model.status == GRB.UNBOUNDED:
        print('The model cannot be solved because it is unbounded')
    elif model.status != GRB.INF_OR_UNBD and model.status != GRB.INFEASIBLE:
        print('Optimization was stopped with status %d' % model.status)
    else:
        # Relax the constraints and try to make the model feasible
        print('The model is infeasible; relaxing the bounds')
        model.feasRelaxS(1, False, False, True)
        model.optimize()

In [21]:
def f_mip(n, output_file=False, print_result=True):
#     failed_regions = []
    
#     def extract_region(model):
#         region = []
#         variables = model.getVars()
#         sol = model.cbGetSolution(variables)
#         for v, val in zip(variables, sol):
#             # only look at the individual indicators
#             if not 'single_ind' in v.varName and not 'component_ind' in v.varName:
#                 continue
#             e = extract_ineq(v)
#             if val == 1 and e is not None:
#                 region.append(e[0])
#             elif val == 0:
#                 # indicator is false
#                 # guaranteed single inequality
#                 region.append(Not(e[0]))
#         return region
    
#     def pos_f_callback(model, where):
#         if where == GRB.Callback.MIPSOL:
#             # Capture node information
#             try:
#                 obj = model.cbGet(GRB.Callback.MIPSOL_OBJ)
#                 print("Current objective:", obj)
#                 if obj > 0:  # Node is infeasible
#                     failed_regions.append(extract_region(model))
#             except gp.GurobiError as e:
#                 print(f"Gurobi error during callback: {e}")
#             except Exception as e:
#                 print(f"Unexpected error during callback: {e}")
    
    h_xyz = read(H_XYZ_CACHE, min_n=1, max_n=n)
    h_yxz = read(H_YXZ_CACHE, min_n=1, max_n=n)
    num_pos_per_n = [len(H_XYZ[i][1]) for i in range(1,n+1)]
    num_neg_per_n = [len(H_YXZ[i][1]) for i in range(1,n+1)]
    num_pos = sum(num_pos_per_n)
    num_neg = sum(num_neg_per_n)
    
    pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
    neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])
    const = sum([coef(i)*(H_XYZ[i][0] - H_YXZ[i][0]) for i in range(1,n+1)])
    
    all_pos_comp_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
    all_neg_comp_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
    
    model = gp.Model(f'f_{n}(x,y,z)-thresh{n}', env=gp.Env())
    model.Params.OutputFlag = 1
    model.Params.LogToConsole = 1
    model.Params.MIPFocus = 1
    
    gx = model.addVar(lb=1, vtype='I', name='x')
    gy = model.addVar(lb=1, vtype='I', name='y')
    gz = model.addVar(lb=1, vtype='I', name='z')
    sp_to_gp = {x: gx, y: gy, z: gz}
    
    pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) \
                for i in all_pos_comp_inds]
    neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) \
                for i in all_neg_comp_inds]
    
    obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)
    
    model.setObjective(obj, GRB.MINIMIZE)
    f_constr = model.addConstr(obj <= 0, name='non_pos_f')
    # constraints for V
    for i, ineq in enumerate(V):
        model.addConstr(to_gp_ineq(ineq, sp_to_gp), name=f'V{i}')
    
    model.optimize()
    
    if print_result:
        print(analyse_result(model))
    
    if output_file:
        model.write(f'f_{n}(x,y,z)-thresh{n}.lp')
    
    return (model.status == GRB.OPTIMAL and \
            model.MIPGap == 0 and \
            eval_f(n, gx.x, gy.x, gz.x) < 0), \
           model

In [22]:
status, model = f_mip(4, print_result=False)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 898 columns and 360 nonzeros
Model fingerprint: 0xedd93d96
Model has 1543 general constraints
Variable types: 0 continuous, 898 integer (895 binary)
Coefficient statistics:
  Matrix range     [8e-04, 1e+00]
  Objective range  [8e-04, 3e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 2e+01]
Presolve added 2344 rows and 1944 columns
Presolve time: 0.02s
Presolved: 2348 rows, 2842 columns, 7222 nonzeros
Presolved model has 1296 SOS constraint(s)
Variable types: 0 continuous, 2842 integer (1543 binary)

Root relaxation: objective -2.375000e-01, 1327 iterations, 0.01 se

In [23]:
# n=3: (8,9,3)
eval_f(4,8,9,3)

0.027160493827160487

In [29]:
eval_f(3,4,5,3)

0.0018518518518518406

In [ ]:
model.getVars()

In [ ]:
eval_f(3,8,9,7)

# LB on h_n(x,y,z) - h_n(y,x,z)

In [25]:
def h_mip(n, output_file=False, print_result=True):
    h_xyz = read(H_XYZ_CACHE, min_n=1, max_n=n)
    h_yxz = read(H_YXZ_CACHE, min_n=1, max_n=n)
    
    all_pos_comp_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
    all_neg_comp_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
    
    model = gp.Model(f'h_{n}(x,y,z)-h_{n}(y,x,z)', env=gp.Env())
    model.Params.OutputFlag = 1
    model.Params.LogToConsole = 1
    model.Params.MIPFocus = 1
    
    gx = model.addVar(lb=1, vtype='I', name='x')
    gy = model.addVar(lb=1, vtype='I', name='y')
    gz = model.addVar(lb=1, vtype='I', name='z')
    sp_to_gp = {x: gx, y: gy, z: gz}
    
    pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) for i in H_XYZ[n][1]]
    neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) for i in H_YXZ[n][1]]
    const = H_XYZ[n][0] - H_YXZ[n][0]
    
    obj = coef(n) * (gp.quicksum([i[0] for i in pos_inds]) - gp.quicksum([i[0] for i in neg_inds]) + const)
    
    model.setObjective(obj, GRB.MINIMIZE)
    # constraints for V
#     model.addConstr(gz <= gx - 1, name='V1')
#     model.addConstr(gx <= gy - 1, name='V2')
    for i, ineq in enumerate(V):
        model.addConstr(to_gp_ineq(ineq, sp_to_gp), name=f'V{i}')
    
    model.optimize()
    
    if print_result:
        print(analyse_result(model))
    
    if output_file:
        model.write(f'h_{n}(x,y,z)-h_{n}(y,x,z).lp')
    
    return (model.status == GRB.OPTIMAL and model.MIPGap == 0), model

In [27]:
status, model = h_mip(4, print_result=False)
model.getVars()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 785 columns and 5 nonzeros
Model fingerprint: 0x6e4bd36c
Model has 1342 general constraints
Variable types: 0 continuous, 785 integer (782 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 8e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 2e+01]
Presolve added 2040 rows and 1680 columns
Presolve time: 0.01s
Presolved: 2043 rows, 2465 columns, 5989 nonzeros
Presolved model has 1120 SOS constraint(s)
Variable types: 0 continuous, 2465 integer (1342 binary)
Found heuristic solution: objective 0.0061728
Found heuristic solutio

[<gurobi.Var x (value 23.0)>,
 <gurobi.Var y (value 29.0)>,
 <gurobi.Var z (value 4.0)>,
 <gurobi.Var single_ind(15*x <= y,) (value -0.0)>,
 <gurobi.Var main_ind(13*x <= 3*y, y < 7*x) (value -0.0)>,
 <gurobi.Var component_ind(13*x <= 3*y,) (value 0.0)>,
 <gurobi.Var component_ind(y < 7*x,) (value 1.0)>,
 <gurobi.Var main_ind(7*x <= y + z, y < 7*x) (value -0.0)>,
 <gurobi.Var component_ind(7*x <= y + z,) (value -0.0)>,
 <gurobi.Var component_ind(y < 7*x,) (value 1.0)>,
 <gurobi.Var single_ind(7*x <= y + z,) (value -0.0)>,
 <gurobi.Var single_ind(7*x <= y - z,) (value -0.0)>,
 <gurobi.Var main_ind(11*x <= 5*y, y < 3*x) (value -0.0)>,
 <gurobi.Var component_ind(11*x <= 5*y,) (value -0.0)>,
 <gurobi.Var component_ind(y < 3*x,) (value 1.0)>,
 <gurobi.Var main_ind(6*x <= 2*y + z, y < 3*x) (value -0.0)>,
 <gurobi.Var component_ind(6*x <= 2*y + z,) (value -0.0)>,
 <gurobi.Var component_ind(y < 3*x,) (value 1.0)>,
 <gurobi.Var main_ind(2*x <= y, y < 3*x, 3*x < y + z) (value -0.0)>,
 <gurobi.Var

In [28]:
model.objVal

-0.021604938271604937

In [ ]:
float(eval_dh(4,8,9,7))

In [ ]:
eval_f(3,4,5,6) + eval_dh(4,25,26,77) + thresh(4)